In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import LabelEncoder
import pickle
import warnings
warnings.simplefilter('ignore')

In [2]:
pickle_dir = "F:/TARGET frontier/TXT34/pickle_data/"
le_save = False
model_save = True

In [3]:
data_dir = "F:/TARGET frontier/TXT34/"
sample_data = pd.DataFrame([])
for i in range(2015,2018+1):
    data_name = "train"+str(i)
    res = pd.read_csv(data_dir+data_name+".csv",encoding="shift-jis")#,na_values=['--']
    sample_data = pd.concat([sample_data,res])
    
#リークしないように慎重に！
data_name = "predict2019"
res = pd.read_csv(data_dir+data_name+".csv",encoding="shift-jis")#,na_values=['--']
predict_len = len(res)
sample_data = pd.concat([sample_data,res])

tail = "fe3"

In [4]:
pd.set_option('display.max_columns', 100)

In [5]:
drop_cols = ["コース区分","多頭出し"]
train_data = sample_data.drop(drop_cols,axis=1)
train_data.shape

(214317, 46)

In [6]:
class Feature_engineering():
    def __init__(self,df):
        self.df = df
        self.l = len(df)
        
    def labelencoding(self,col):
        le = LabelEncoder()
        le.fit(self.df[col])
        if le_save:
            with open(pickle_dir+tail+col+'.pickle', mode='wb') as fp:
                pickle.dump(le, fp)
        self.df[col] = le.transform(self.df[col])
    
    def more_than_double_le(self,lst):
        a,b = lst[0],lst[1]
        le = LabelEncoder()
        le.fit(pd.concat([self.df[a],self.df[b]]))
        if le_save:
            with open(pickle_dir+tail+a+b+'.pickle', mode='wb') as fp:
                pickle.dump(le, fp)
        self.df[a] = le.transform(self.df[a])
        self.df[b] = le.transform(self.df[b])
        
    def handle_nan(self,col):
        self.df.fillna({col:"--"},inplace=True)
        
    def fill_nan(self,col):
        self.df.fillna({col:"0"},inplace=True)
    
    def handle_mark(self,col):
        self.df[col] = self.df[col].str.replace("△","")
        self.df[col] = self.df[col].str.replace("▲","")
        self.df[col] = self.df[col].str.replace("☆","")
        #["△","▲","☆"]
    
    def handle_class(self,col):
        dic_class = {"500万":"1勝","1000万":"2勝","1600万":"3勝","OP(L)":"ｵｰﾌﾟﾝ","重賞":"Ｇ３","(不明)":"--"}
        self.df[col].replace(dic_class,inplace=True)
        
    def handle_specific_nan(self,col):
        dic_specific = {"----":np.nan}
        self.df[col].replace(dic_specific,inplace=True)
    
    def drop_row(self,col):
        self.df.dropna(inplace=True,subset = [col])
        self.df.reset_index(drop=True,inplace=True)
    
    def handle_abs(self,col):
        self.df[col] = self.df[col].astype("float64")
        self.df[col] = self.df[col].where(self.df[col]>0 , 0) #0より小さいなら0を入れてやる
    
    def set_validation(self,y_col,ozz_col):
        self.df[ozz_col] = self.df[ozz_col].astype("float64")
        self.l = len(self.df)
        res = np.array([0.0]*self.l)
        
        for i in range(self.l):
            res[i] = (1/math.exp(3*self.df[y_col][i]))*(1/math.exp(1/self.df[ozz_col][i]*5)) #2/math.exp(1/i*3) min(50,self.df[ozz_col][i]) #1/math.exp(3*self.df[y_col][i])
        self.df[y_col] = np.round(res,5)
    
    #わざと着差をいじる
    def essence(self,y_col,ozz_col):
        res = []
        for i,j in zip(self.df[y_col],self.df[ozz_col]):
            if i == 0 and 9 <= j <= 13:
                res.append(i-0.3)
            else:
                res.append(i)
        self.df[y_col] = pd.DataFrame(res)
    
    def ranking_and_number(self,ID_col):
        ranking = []
        number = []
        num = 0
        rank = 1
        res = ""
        for i,j in enumerate(self.df[ID_col]):
            j = str(j)
            if i == 0:
                ranking.append(rank)
                rank += 1
                res = j[:-2]
                num += 1
                continue
            if res != j[:-2]:
                res = j[:-2]
                number += [num]*num
                rank = 1
                ranking.append(rank)
                rank += 1
                num = 1
            else:
                num += 1
                ranking.append(rank)
                rank += 1
        number += [num]*num
        print(len(number))
        print(len(ranking))
        self.df = pd.concat([self.df,pd.Series(ranking,name="順位"),pd.Series(number,name="頭数")],axis=1)
        
    def basyo_kyori_sibadart(self,basyo,kyori,sibadart):
        self.l = len(self.df)
        res = []
        for i in range(self.l):
            if i%10000==0:
                print(i)
            b = self.df[basyo][i]
            k = self.df[kyori][i]
            s = self.df[sibadart][i]
            if any(j == np.nan for j in [b,k,s]):
                res.append(np.nan)
            else:
                res.append(str(b)+str(k)+str(s))
        le = LabelEncoder()
        le.fit(res)
        res = le.transform(res)
        name = basyo+"距離芝ダ"
        res = pd.Series(res,name=name)
        self.df =  pd.concat([self.df,res],axis=1)
    
    def convert_kyakusitu(self,col): #labelencodeより先にやること
        self.l = len(self.df)
        for i in range(self.l):
            if self.df[col][i] == "逃":
                self.df[col][i] = "逃げ"
            elif self.df[col][i] == "先":
                self.df[col][i] = "先行"
            elif self.df[col][i] == "中":
                self.df[col][i] = "中団"
            elif self.df[col][i] == "後":
                self.df[col][i] = "後方"
            elif self.df[col][i] == "マ":
                self.df[col][i] = "ﾏｸﾘ" #半角注意
        
        


In [7]:
fe = Feature_engineering(train_data)

In [8]:
nan_cols = ["父タイプ名","前走場所","前芝・ダ","前クラス名","前走馬場状態","前走脚質","前好走","替","所属","前騎手","調教師"]
for i in nan_cols:
    print("processing:",i)
    fe.handle_nan(i)

processing: 父タイプ名
processing: 前走場所
processing: 前芝・ダ
processing: 前クラス名
processing: 前走馬場状態
processing: 前走脚質
processing: 前好走
processing: 替
processing: 所属
processing: 前騎手
processing: 調教師


In [9]:
fill_nan_with_0 = ["複勝配当"]
for i in fill_nan_with_0:
    print("processing:",i)
    fe.fill_nan(i)

processing: 複勝配当


In [10]:
mark_cols = ["斤量","前走斤量"]
for i in mark_cols:
    print("processing:",i)
    fe.handle_mark(i)

processing: 斤量
processing: 前走斤量


In [11]:
h_class = ["クラス名","前クラス名"]
for i in h_class:
    print("processing:",i)
    fe.handle_class(i)

processing: クラス名
processing: 前クラス名


In [12]:
double_le = [["場所","前走場所"],["芝・ダ","前芝・ダ"],["馬場状態","前走馬場状態"],["クラス名","前クラス名"],["騎手","前騎手"]]
for i in double_le:
    print("processing:",i)
    fe.more_than_double_le(i)

processing: ['場所', '前走場所']
processing: ['芝・ダ', '前芝・ダ']
processing: ['馬場状態', '前走馬場状態']
processing: ['クラス名', '前クラス名']
processing: ['騎手', '前騎手']


In [13]:
cat_cols = ["性別","種牡馬","父タイプ名","前走脚質","前好走","替","所属","調教師"]
for i in cat_cols:
    print("processing:",i)
    fe.labelencoding(i)

processing: 性別
processing: 種牡馬
processing: 父タイプ名
processing: 前走脚質
processing: 前好走
processing: 替
processing: 所属
processing: 調教師


In [14]:
specific_nan = ["前走着差タイム","着差"]
for i in specific_nan:
    print("processing",i)
    fe.handle_specific_nan(i)

processing 前走着差タイム
processing 着差


In [15]:
drop_na = ["単勝オッズ"]
for i in drop_na:
    print("processing",i)
    fe.drop_row(i)

processing 単勝オッズ


In [16]:
handle_abs = ["前走着差タイム","着差"]
for i in handle_abs:
    print("processing:",i)
    fe.handle_abs(i)

processing: 前走着差タイム
processing: 着差


In [17]:
"""essence = [["着差","単勝オッズ"]]
for i,j in essence:
    print("processing:",i,j)
    fe.essence(i,j)"""

'essence = [["着差","単勝オッズ"]]\nfor i,j in essence:\n    print("processing:",i,j)\n    fe.essence(i,j)'

In [18]:
#順位、頭数は既に組み込み済み
fe.ranking_and_number("レースID(旧)")
train_data = fe.df #これでranking_and_numberが反映される

214317
214317


In [19]:
bks = [["場所","距離","芝・ダ"],["前走場所","前距離","前芝・ダ"]]
for b,k,s in bks:
    print("processing:",b,k,s)
    fe.basyo_kyori_sibadart(b,k,s)
    train_data = fe.df

processing: 場所 距離 芝・ダ
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
processing: 前走場所 前距離 前芝・ダ
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000


In [20]:
#事故った時用にtrain_dataをdf型で保存しておく
if model_save:
    with open(pickle_dir+tail+"train_data"+'.pickle', mode='wb') as fp:
        pickle.dump(train_data, fp)

In [21]:
all_len = len(train_data)
all_len

214317

In [22]:
train = train_data.iloc[:all_len-predict_len-1]
predict = train_data.iloc[all_len-predict_len:]

In [23]:
train.to_csv(data_dir+"train"+tail+".csv",index=False,encoding="shift-jis")
predict.to_csv(data_dir+"predict"+tail+".csv",index=False,encoding="shift-jis")

In [ ]:
"""with open(pickle_dir+tail+"train_data"+'.pickle', mode='rb') as fp:
    train_data = pickle.load(fp)"""

## ここからID検索、前5走取得、データフレーム変換  
最初に、train_dataのindexをレースIDにする。  
大本のデータフレーム名(実際にxgmachinaに使うもの)をbase_dfとする。  
resを各レース情報とし、ここからres_dfにひとつずつ追加していく。  
idファイルを読み込み、該当idのレースを引っ張ってきてres_dfに追加していく。  
res_dfが完成したらbase_dfに追加し、res_dfを初期化する。  

In [ ]:
len(train_data)

In [ ]:
train_data = train_data.set_index("レースID(旧)")

In [ ]:
i = 2015
id_df = pd.read_csv(data_dir+"id"+f"{i}"+".csv",encoding="shift-jis",names=("日付データ","今走","前1走","前2走","前3走","前4走","前5走"))

In [ ]:
train_data.iloc[0][["騎手","前騎手"]]

In [ ]:
columns_name = ["レースID(旧)","場所","芝・ダ","距離","クラス名","馬場状態","頭数","騎手","枠番","性別","年齢","キャリア","斤量","間隔","休み明け〜戦目","種牡馬","父タイプ名","所属","調教師","確定着順","着差","単勝オッズ","複勝オッズ下限",
                "複勝オッズ上限","複勝配当","前走レースID(旧)","前走場所","前芝・ダ","前距離","前クラス名","前走馬場状態","前走出走頭数","替","前騎手","前走枠番","前走脚質","前走斤量","前走Ave-3F","前走上り3F","前走PCI3",
                "前走RPCI","前PCI","前好走","前走確定着順","前走着差タイム","前走単勝オッズ","場所距離芝ダ","前走場所距離芝ダ","前走レースID(旧)1","前走場所1","前芝・ダ1","前距離1","前クラス名1","前走馬場状態1","前走出走頭数1",
                "替1","前騎手1","前走枠番1","前走脚質1","前走斤量1","前走Ave-3F1","前走上り3F1","前走PCI31","前走RPCI1","前PCI1","前好走1","前走確定着順1","前走着差タイム1","前走単勝オッズ1","場所距離芝ダ1","前走場所距離芝ダ1",
                "前走レースID(旧)2","前走場所2","前芝・ダ2","前距離2","前クラス名2","前走馬場状態2","前走出走頭数2","替2","前騎手2","前走枠番2","前走脚質2","前走斤量2","前走Ave-3F2","前走上り3F2","前走PCI32","前走RPCI2",
                "前PCI2","前好走2","前走確定着順2","前走着差タイム2","前走単勝オッズ2","場所距離芝ダ2","前走場所距離芝ダ2","前走レースID(旧)3","前走場所3","前芝・ダ3","前距離3","前クラス名3","前走馬場状態3","前走出走頭数3",
                "替3","前騎手3","前走枠番3","前走脚質3","前走斤量3","前走Ave-3F3","前走上り3F3","前走PCI33","前走RPCI3","前PCI3","前好走3","前走確定着順3","前走着差タイム3","前走単勝オッズ3","場所距離芝ダ3","前走場所距離芝ダ3",
                "前走レースID(旧)4","前走場所4","前芝・ダ4","前距離4","前クラス名4","前走馬場状態4","前走出走頭数4","替4","前騎手4","前走枠番4","前走脚質4","前走斤量4","前走Ave-3F4","前走上り3F4","前走PCI34","前走RPCI4",
                "前PCI4","前好走4","前走確定着順4","前走着差タイム4","前走単勝オッズ4","場所距離芝ダ4","前走場所距離芝ダ4","前走レースID(旧)5","前走場所5","前芝・ダ5","前距離5","前クラス名5","前走馬場状態5","前走出走頭数5","替5",
                "前騎手5","前走枠番5","前走脚質5","前走斤量5","前走Ave-3F5","前走上り3F5","前走PCI35","前走RPCI5","前PCI5","前好走5","前走確定着順5","前走着差タイム5","前走単勝オッズ5","場所距離芝ダ5","前走場所距離芝ダ5",
               ]

In [ ]:
train_data = train_data.to_dict(orient="index")

In [ ]:
s = id_df.iloc[33000][1]
print(int(s))

In [ ]:
for i,j in enumerate(train_data.keys()):
    if 32999 <= i <= 33001:
        print(j)

In [ ]:
import time
t1 = 0
t2 = 0

In [ ]:
base_df = pd.DataFrame(columns=columns_name)
e1 = []
e2 = []
for i in range(2015,2019+1):
    id_df = pd.read_csv(data_dir+"id"+f"{i}"+".csv",encoding="shift-jis",names=("日付データ","今走","前1走","前2走","前3走","前4走","前5走"))
    id_df = id_df.values.tolist()
    res_df = pd.DataFrame(columns=columns_name)
    res_res_df = pd.DataFrame(columns=columns_name)
    for j in range(len(id_df)):
        if j%1000==0:
            t2 = time.time()
            print("processing:",i,j,t2-t1)
            t1 = t2
        for k in range(6):
            if k == 0: #今走の場合のみ、今走のデータを取り出す
                try:
                    race_id = id_df[j][k+1]
                    res = pd.Series({"レースID(旧)":race_id})
                    res = res.append(pd.Series(train_data[race_id]))
                    continue
                except: #ここでエラー出たらもうしょうがないので飛ばす
                    try:
                        race_id = int(id_df[j][k+1])
                        res = pd.Series({"レースID(旧)":race_id})
                        res = res.append(pd.Series(train_data[race_id]))
                        continue
                    except:
                        e1.append(race_id)
                        break
            race_id = id_df[j][k+1]
            #race_id = id_df.iloc[j][f"前{k}走"]
            if race_id != np.nan:
                try:
                    race_data = pd.Series(train_data[race_id])
                    race_data = race_data[["前走レースID(旧)","前走場所","前芝・ダ","前距離","前クラス名","前走馬場状態","前走出走頭数","替",
                                                         "前騎手","前走枠番","前走脚質","前走斤量","前走Ave-3F","前走上り3F","前走PCI3","前走RPCI","前PCI",
                                                         "前好走","前走確定着順","前走着差タイム","前走単勝オッズ","場所距離芝ダ","前走場所距離芝ダ"]]
                    race_data = race_data.rename(index=lambda s: s+str(k))
                    res = res.append(race_data) #必要なデータだけで良い
                except:
                    break
            else: #レースIDが欠損してた場合
                break
        
        res.name = j#res:馬毎
        res_df = res_df.append(res) #res_df:年毎
        if j%500==0:
            res_res_df = pd.concat([res_res_df,res_df])
            res_df = pd.DataFrame(columns=columns_name)
    #最後の処理(i%500でひっかからない最後)
    res_res_df = pd.concat([res_res_df,res_df])
    res_df = pd.DataFrame(columns=columns_name)
    
    base_df = pd.concat([base_df,res_res_df])
    res_res_df = pd.DataFrame(columns=columns_name)
    #base_df = base_df.append(res_res_df) #base_df:全データ


In [ ]:
with open(pickle_dir+tail+"base_df"+'.pickle', mode='wb') as fp:
        pickle.dump(base_df, fp)

In [ ]:
"""with open(pickle_dir+tail+"base_df"+'.pickle', mode='rb') as fp:
    base_df = pickle.load(fp)"""

all_len : 2013-2019年までの全データ長  
train_all_len : 2015-2019までのデータ長  
train_len : 2015-2018まで  
predict_len : 2019年のデータ長  

In [ ]:
all_len = len(train_data)
all_len

In [ ]:
train_all_len = len(base_df)

In [ ]:
train_all_len

In [ ]:
predict_len

In [ ]:
train = base_df.iloc[:train_all_len-predict_len-1]

In [ ]:
predict = base_df.iloc[train_all_len-predict_len:]

In [ ]:
len(train)

In [ ]:
len(predict) #predoct_lenと一致しているか確認！

In [ ]:
if model_save:
    with open(pickle_dir+tail+"train"+'.pickle', mode='wb') as fp:
        pickle.dump(train, fp)
if model_save:
    with open(pickle_dir+tail+"predict"+'.pickle', mode='wb') as fp:
        pickle.dump(predict, fp)

In [ ]:
train.to_csv(data_dir+"train"+tail+".csv",index=False,encoding="shift-jis")
predict.to_csv(data_dir+"predict"+tail+".csv",index=False,encoding="shift-jis")